In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_colwidth=100

In [2]:
# Shingle generators 
# Arguments : Message string, shingle size {in words}
# Returns : All shingles formed with k words
def shingle_generator(message, k):
    message = message.split(' ')
    item_set = []
    for i in range(0, len(message)-k):
        shingle = ""
        for j in range(0, k):
            shingle = shingle + message[i+j] + " "
        item_set.append(shingle)
    return item_set

In [3]:
# dump pkl into a pandas dataframe. 
data = pd.read_pickle('data/string_study.pkl')

In [9]:
data["word_count"] = data._info.apply(lambda x:len(x.split(' ')))
data.head()

,atime,task,deployment,raw,dtime,msg,_info,word_count
0,1524387149611,attach-volume,wig_project_003,2018-04-22 10:52:29.611 17979 ERROR rallytester.rallytester [-] [nova attach-volume wig_project_...,2018-04-22 08:00:00,waiting for Server to become ('ACTIVE'),Rally tired waiting 1440.00 seconds for Server rally-9db3-aPln:176b6fd3-8a2d-4b60-a5bd-73575c4ad...,14
1,1524387185889,boot-from-volume-linux,gva_shared_016,2018-04-22 10:53:05.889 25667 ERROR rallytester.rallytester [-] [nova boot-from-volume-linux gva...,2018-04-22 08:00:00,"Quota exceeded for cores, instances: Requested ..... cores, instances (HTTP 403) (Request-ID: re...","Quota exceeded for cores, instances: Requested 1, 1, but already used 10, 10 of 10, 10 cores, in...",22
2,1524387196073,boot-linux,gva_shared_016,2018-04-22 10:53:16.073 25840 ERROR rallytester.rallytester [-] [nova boot-linux gva_shared_016]...,2018-04-22 08:00:00,"Quota exceeded for cores, instances: Requested ..... cores, instances (HTTP 403) (Request-ID: re...","Quota exceeded for cores, instances: Requested 1, 1, but already used 10, 10 of 10, 10 cores, in...",22
3,1524387321429,boot-linux,gva_project_013,2018-04-22 10:55:21.429 19322 ERROR rallytester.rallytester [-] [nova boot-linux gva_project_013...,2018-04-22 08:00:00,waiting for Server to become ('ACTIVE'),Rally tired waiting 1440.00 seconds for Server rally-15fc-N4mw:77640b75-df94-4e6d-91d6-3338fffcd...,14
4,1524387330897,attach-volume,gva_project_023,2018-04-22 10:55:30.897 19363 ERROR rallytester.rallytester [-] [nova attach-volume gva_project_...,2018-04-22 08:00:00,waiting for Server to become ('ACTIVE'),Rally tired waiting 1440.00 seconds for Server rally-1c07-9a14:c7c5f878-2495-45bf-873d-da11b5d88...,14


In [8]:
msg_array = np.array(data._info.copy())
msg_array

array(["Rally tired waiting 1440.00 seconds for Server rally-9db3-aPln:176b6fd3-8a2d-4b60-a5bd-73575c4ad161 to become ('ACTIVE') current status BUILD",
       'Quota exceeded for cores, instances: Requested 1, 1, but already used 10, 10 of 10, 10 cores, instances (HTTP 403) (Request-ID: req-23b5fe81-0bab-4f42-84e9-97e3e6847c04)',
       'Quota exceeded for cores, instances: Requested 1, 1, but already used 10, 10 of 10, 10 cores, instances (HTTP 403) (Request-ID: req-f1b55ece-6bd1-475b-9692-7e443d44d52b)',
       ...,
       "Rally tired waiting for Host ip:128.142.149.18 to become ('ICMP UP') current status ICMP DOWN",
       'Resource <Server: rally-0795-TZCm> has ERROR status. Fault: {u\'message\': u"Request Failed: internal server error while processing your request.\\nNeutron server returns request_ids: [\'req-e81f3444-c2fc-4456-ac9c-eba09662512a\']", u\'code\': 500, u\'created\': u\'2018-04-05T22:50:03Z\'}',
       'Rally result is not a JSON. Cannot process results. <sep-rally-s

In [12]:
class_by_len_set = set(data.word_count)

In [13]:
class_by_len_set

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 31,
 32,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 72,
 73,
 81,
 118,
 749}

In [15]:
# Taking the smallest one to test minhash algo
len(data.query("word_count == 19"))

7

In [16]:
filtered_data = data.query("word_count == 19")

In [17]:
raw_msgs = filtered_data._info

In [18]:
raw_msgs

19595    Resource <Server: rally-03d2-Pq0H> has ERROR status.  Fault: {u'message': u'internal error: clie...
19683    Resource <Server: rally-a7ad-kpUm> has ERROR status.  Fault: {u'message': u'internal error: clie...
23778    Resource <Server: rally-97fb-wWlm> has ERROR status.  Fault: {u'message': u'internal error: clie...
24621    Resource <Server: rally-119c-PZVt> has ERROR status.  Fault: {u'message': u'internal error: clie...
25940    Resource <Server: rally-d0e6-KIQi> has ERROR status.  Fault: {u'message': u'internal error: clie...
26298    Resource <Server: rally-600e-FYGn> has ERROR status.  Fault: {u'message': u'internal error: clie...
34443    Rally result is not a JSON. Cannot process results. <sep-rally-sep> Rally result is not a JSON. ...
Name: _info, dtype: object

In [28]:
# raw_msgs = pd.DataFrame(raw_msgs, columns=["_info"])

In [47]:
# Generate the set of words of all raw messages
message_set = []
%time

for item in np.array(raw_msgs):
#     print(item)
    item = str(item[0])
    set_of_words = set(item.split(' '))
    # Shingle size is set to 3 words
    set_of_words = set(shingle_generator(item, 1))
    message_set.append(set_of_words)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs


In [49]:
import numpy as np
import csv
import pandas
from datasketch import MinHash, MinHashLSH

In [50]:
# Generate MinHash instances for all messages
m = []
for i in range (0, len(message_set)):
    m_temp = MinHash(num_perm=128)
    m.append(m_temp)
len(m)

7

In [55]:
i = 0
for message in message_set:
    for word in message:
        m[i].update(word.encode('utf-8'))
    i = i + 1

In [56]:
# Create an LSH index 
# %time
lsh = MinHashLSH(threshold=0.01, num_perm=128)
count = 0
for i in range (0, len(m)):
    string = "m" + str(i)
    lsh.insert(string, m[i])
    count += 1
print (count)

7


In [60]:
# Find all the messages similar to the fist message with a specified threshold
%time
result = lsh.query(m[1])
(result)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.82 µs


['m0', 'm3', 'm2', 'm4', 'm5', 'm6', 'm1']